This should describe whole process to calculate incidences using pandas library.

first we have to load datas and definitions

In [2]:
import numpy as np
import pandas as pd

#definitions
world_std = [12,10,9,9,8,8,6,6,6,6,5,4,4,3,2,1,.5,.5] 
agegroupdef = {
            "0-4": 1,"5-9": 2,"10-14": 3,"15-19": 4,"20-24": 5,"25-29": 6,"30-34": 7,"35-39": 8,"40-44": 9,
            "45-49": 10,"50-54": 11,"55-59": 12,"60-64": 13,"65-69": 14,"70-74": 15,"75-79": 16,"80-84": 17, "85+":18,
            }



In [192]:
#calculation specific definitions
agegroups = [1,18] # = All. You can limit using start and end key of "agegroupdef"
periods = [[1998,2002],[2001,2005],[2004,2008],[2007,2012]]
gender = 2 # 0 Male, 1 Female
unitratio = 100000 # res: 1/100000 persons

# population data import
popdt = pd.read_csv("Population_Ragusa_Caltanissetta.csv")


In [54]:
# cancer cases import
casedt = pd.read_csv("C713.csv",header=None)

In [55]:
casedt.columns = ['cntryid','year','sex','agegroup','municid','cases']


In [250]:
# municipality list: All
r_final = popdt[['municid','sex']].groupby(['municid']).count().reset_index()
r_final['sex'] = gender
r_final.set_index('municid')
popdt

,cntryid,year,sex,agegroup,municid,pop
0,38,1998,0,1,88001,273
1,38,1998,0,2,88001,238
2,38,1998,0,3,88001,288
3,38,1998,0,4,88001,288
4,38,1998,0,5,88001,298
...,...,...,...,...,...,...
18355,38,2012,1,14,85022,44
18356,38,2012,1,15,85022,53
18357,38,2012,1,16,85022,65
18358,38,2012,1,17,85022,48


In [208]:
# preparations
# wstd > dataframe with index for pd.join
wstd = pd.DataFrame({'wstd': world_std})
s = pd.DataFrame(index=list(range(1,18)))
wstdi = s.join(wstd)

# municipality list: All
#r_final = popdt[['municid','sex']].groupby('municid').agg('sex').reset_index()

# aggregate the data to have agegroups as columns
popdt = popdt[(popdt.sex == gender)] if gender < 2 else popdt
casedt = casedt[(casedt.sex == gender)] if gender < 2 else casedt
casedt.agegroup = casedt.agegroup+1 # Sic datan fix: agegroup min 0

# here is supposed that data have all 18 agegroups defined
agsascols = popdt[['year','municid','agegroup','pop']].groupby(['year','municid','agegroup']).sum('pop').unstack(level=-1)
nd = agsascols.add_prefix('a_')
nd = nd.a_pop.reset_index()



In [297]:
resultsp = {}
resultsc = {}
tmpres = {}

# municipality list: All
r_final = popdt[['municid']].groupby(['municid']).count().reset_index()
r_final['sex'] = gender
#r_final.set_index('municid')

i = 0
for yper in periods:
    print(f"Start: {str(yper[0])} - End: {str(yper[1])}" )
    keynm = "k_" +str(i) #results[yper[0]] = []
    pop_by_period = popdt[(popdt.year >= yper[0]) & (popdt.year <= yper[1])]
    case_by_period = casedt[(casedt.year >= yper[0]) & (casedt.year <= yper[1])]
    resultsp[keynm] = pop_by_period[['municid','agegroup','pop']].groupby(['municid','agegroup']).sum('pop')
    #casesum[yper[0]] 
    resultsc[keynm] = case_by_period[['municid','agegroup','cases']].groupby(['municid','agegroup']).sum('cases')
    
    
    # Merge pop and cases correspondig rows by municid and agegroup
    merged_res = resultsp[keynm].merge(resultsc[keynm], left_on=['municid', 'agegroup'], right_index=True)
    
    # Add agestd weight by age group using wstdi (pd.dataframe)
    r = merged_res.merge(wstdi, left_on=['agegroup'], right_index=True).reset_index()
    
    # calc age stantard rate
    #numpy: rate = sum(np_r[:,1] * np_r[:,2] / np_r[:,0])*100000
    # pandas: 
    r['rateweights'] = r['cases'] * r['wstd'] / r['pop'] * unitratio # result: 1/100000 persons
    r_grp = r[['municid','rateweights']].groupby('municid').sum('rateweights').reset_index()
    #tmpres[keynm] = r_grp
    # result should look like: municid, period1,...,periodN
    fieldname = f"{str(yper[0])}-{str(yper[0])}"
    #r_grp.rename({'rateweights':fieldname}, axis='columns')
    r_grp = pd.DataFrame({ fieldname: list(r_grp['rateweights'])},index=list(r_grp['municid']))
    tmpres[keynm] = r_grp
    r_final = r_final.set_index('municid',drop=False).join(r_grp,lsuffix='_2')
    #r_final.set_index('municid')
    i += 1

# reset index and replace Nans with zero (no value here means zero value, it has meaning, because 
# we know that it is not measeure problem but real situation, where no cases exist
r_final=r_final.rename({'municid': 'municid_'},axis='columns')
r_final.reset_index().fillna(0)


Start: 1998 - End: 2002
Start: 2001 - End: 2005
Start: 2004 - End: 2008
Start: 2007 - End: 2012


,municid,municid_,sex,1998-1998,2001-2001,2004-2004,2007-2007
0,85001,85001,2,0.000000,0.000000,0.000000,0.000000
1,85002,85002,2,0.000000,0.000000,0.000000,0.000000
2,85003,85003,2,0.000000,0.000000,0.000000,0.000000
3,85004,85004,2,0.000000,0.000000,29.405999,25.339978
4,85005,85005,2,0.000000,0.000000,0.000000,0.000000
5,85006,85006,2,0.000000,0.000000,0.000000,0.000000
6,85007,85007,2,0.000000,0.000000,0.000000,54.321490
7,85008,85008,2,0.000000,0.000000,0.000000,0.000000
8,85009,85009,2,0.000000,0.000000,0.000000,0.000000
9,85010,85010,2,0.000000,0.000000,0.000000,0.000000


In [293]:
#r_f=r_final[['municid','sex']]
#r_f.set_index('municid', drop=False)
#r_f=r_f.join(tmpres['k_1'])
#tmpres['k_0']
#r_f
r_final=r_final.rename({'municid': 'municid_'},axis='columns')


In [295]:
r_final.reset_index()


,municid,municid_,sex,1998-1998,2001-2001,2004-2004,2007-2007
0,85001,85001,2,NaN,NaN,NaN,NaN
1,85002,85002,2,NaN,NaN,NaN,NaN
2,85003,85003,2,NaN,NaN,NaN,NaN
3,85004,85004,2,NaN,NaN,29.405999,25.339978
4,85005,85005,2,NaN,NaN,NaN,NaN
5,85006,85006,2,NaN,NaN,NaN,NaN
6,85007,85007,2,NaN,NaN,NaN,54.321490
7,85008,85008,2,NaN,NaN,NaN,NaN
8,85009,85009,2,NaN,NaN,NaN,NaN
9,85010,85010,2,NaN,NaN,NaN,NaN


In [13]:
nd.sex

0       0
1       0
2       0
3       0
4       0
       ..
1015    1
1016    1
1017    1
1018    1
1019    1
Name: sex, Length: 1020, dtype: int64

In [33]:
popdt[(popdt.year >= yper[0] & popdt.year <= yper[1])]

,cntryid,year,sex,agegroup,municid,pop
0,38,1998,0,1,88001,273
1,38,1998,0,2,88001,238
2,38,1998,0,3,88001,288
3,38,1998,0,4,88001,288
4,38,1998,0,5,88001,298
...,...,...,...,...,...,...
18337,38,2012,0,14,85022,53
18338,38,2012,0,15,85022,36
18339,38,2012,0,16,85022,45
18340,38,2012,0,17,85022,29


In [51]:
popdt.iloc[1:4,3:6].sum().reset_index()




,index,0
0,agegroup,9
1,municid,264003
2,pop,814


In [9]:
k = "k_0"
r={}
r[k]=[1,2,3]

In [181]:
r_final

,sex
municid,
85001,540
85002,540
85003,540
85004,540
85005,540
85006,540
85007,540
85008,540
85009,540


In [21]:
resultsc['k_1'].reset_index()


,municid,agegroup,cases
0,88003,1,1
1,88008,1,1
2,88009,1,2


In [67]:
#pd.concat(resultsp['k_0'].reset_index(),resultsc['k_0'].reset_index(), axis=1)
merged_res = resultsp['k_0'].merge(resultsc['k_0'], left_on=['municid', 'agegroup'], right_index=True)

In [98]:
merged_res

,,pop,cases
municid,agegroup,,
88001,2,2580,1
88002,2,2312,1
88008,2,5392,1
88011,1,6591,1
88012,2,18294,1


In [103]:
wstd = pd.DataFrame({'wstd': world_std})
s = pd.DataFrame(index=list(range(1,18)))
wstdi = s.join(wstd)
wstdi

,wstd
1,10.0
2,9.0
3,9.0
4,8.0
5,8.0
6,6.0
7,6.0
8,6.0
9,6.0
10,5.0


In [105]:
r = merged_res.merge(wstdi, left_on=['agegroup'], right_index=True).reset_index()

In [130]:
# numpy conversion not needed
#np_r = np.array(r[['municid','pop','cases','wstd']])
#np_r_by_row = (np_r[:,2] * np_r[:,3] / np_r[:,1])
#r['rateweights'] = np_r_by_row
r['rateweights'] = r['cases'] * r['wstd'] / r['pop']

In [134]:
# changing value of data (just for test)
r.loc[r.municid == 88008,'municid'] = 88002


In [141]:
r[['municid','rateweights']].groupby('municid').sum(['rateweights'])

,rateweights
municid,
88001,0.003488
88002,0.005562
88011,0.001517
88012,0.000492


In [139]:
r

,municid,agegroup,pop,cases,wstd,rateweights
0,88001,2,2580,1,9.0,0.003488
1,88002,2,2312,1,9.0,0.003893
2,88002,2,5392,1,9.0,0.001669
3,88012,2,18294,1,9.0,0.000492
4,88011,1,6591,1,10.0,0.001517


In [142]:
popdt

,cntryid,year,sex,agegroup,municid,pop
0,38,1998,0,1,88001,273
1,38,1998,0,2,88001,238
2,38,1998,0,3,88001,288
3,38,1998,0,4,88001,288
4,38,1998,0,5,88001,298
...,...,...,...,...,...,...
18355,38,2012,1,14,85022,44
18356,38,2012,1,15,85022,53
18357,38,2012,1,16,85022,65
18358,38,2012,1,17,85022,48


In [157]:
r_final = popdt[['municid']].groupby('municid').count() 
tmp = pd.DataFrame({'rateweights': list(r['rateweights'])},index=r['municid'])
#[['municid','rateweights']]
r_final.join(tmp)

,rateweights
municid,
85001,NaN
85002,NaN
85003,NaN
85004,NaN
85005,NaN
85006,NaN
85007,NaN
85008,NaN
85009,NaN


In [148]:
r


,municid,agegroup,pop,cases,wstd,rateweights
0,88001,2,2580,1,9.0,0.003488
1,88002,2,2312,1,9.0,0.003893
2,88002,2,5392,1,9.0,0.001669
3,88012,2,18294,1,9.0,0.000492
4,88011,1,6591,1,10.0,0.001517


In [151]:
tmp


,rateweights
municid,
88001,NaN
88002,NaN
88002,NaN
88012,NaN
88011,NaN


In [156]:
pd.DataFrame({'name': list(r['rateweights'])},index=r['municid'])

,name
municid,
88001,0.003488
88002,0.003893
88002,0.001669
88012,0.000492
88011,0.001517


In [188]:
popdt[['municid','sex']].groupby('municid').

TypeError: Must provide 'func' or tuples of '(column, aggfunc).